In [1]:
!pip install torchtext==0.11.2

     |████████████████████████████████| 8.0 MB 4.7 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.2 MB/s eta 0:00:40tcmalloc: large alloc 1147494400 bytes == 0x3a8d2000 @  0x7f10328d4615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████████████████████████████████| 881.9 MB 1.8 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.12.0
    Uninstalling torchtext-0.12.0:
      Successfully uninstalled torchtext-0.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of

In [2]:
import os
import sys
import torch
from torch.nn import functional as F
import numpy as np
from torchtext import data
from torchtext import datasets
from torchtext.vocab import Vectors, GloVe


In [3]:
from torchtext.vocab import GloVe

glove = GloVe(name="twitter.27B", dim=25)

# glove.twitter.27B.25d

# glove = GloVe(name='6B')


.vector_cache/glove.twitter.27B.zip: 1.52GB [04:46, 5.31MB/s]                            
100%|█████████▉| 1193513/1193514 [00:23<00:00, 51742.07it/s]


In [4]:
## Read in data
import pandas as pd 

df_train = pd.read_csv("hate_train.csv")
df_test = pd.read_csv("hate_test.csv")
df_val = pd.read_csv("hate_val.csv")

print(df_train.shape)
df_train.head()

(22373, 9)


,Unnamed: 0,Unnamed: 0.1,id,label,tweet,hash_tag,clean_tweet,tokenized_tweet,tokenized_tweet_NLTK
0,27857,27857,27858,0,"omg. omg. omg. yay! i found it, and at a wond...","['segasaturn', 'throwbackâ']",omg omg omg yay found wonderful price segasatu...,"omg. omg. omg. yay! i found it, and at a wond...",omg omg omg yay found wonderful price segasatu...
1,31205,31205,31206,0,#payintheusa polar bear climb racing: angry ...,['payintheusa'],payintheusa polar bear climb racing angry pola...,<hashtag> payintheusa <elong>polar bear climb...,payintheusa polar bear climb racing angry pola...
2,8440,8440,8441,0,#trainhard polar bear climb racing: angry po...,['trainhard'],trainhard polar bear climb racing angry polar ...,<hashtag> trainhard <elong>polar bear climb r...,trainhard polar bear climb racing angry polar ...
3,5005,5005,5006,1,he should turn in his resignation.,[],turn resignation,he should turn in his resignation.,turn resignation
4,3898,3898,3899,0,ððð . . happy bihday!! to hajime hoso...,"['bihday', '30æ', 'ã']",happy bihday hajime hosogai bihday bihday 30,ððð . . happy bihday! <repeat> to haj...,. . happy bihday hajime hosogai . . . bihday b...


In [70]:
df_train = df_train[df_train['clean_tweet'].notna()]
df_test = df_test[df_test['clean_tweet'].notna()]
df_val = df_val[df_val['clean_tweet'].notna()]

df_train[["clean_tweet", "label"]].to_csv("train.csv")
df_test[["clean_tweet", "label"]].to_csv("test.csv")
df_val[["clean_tweet", "label"]].to_csv("val.csv")

In [69]:
df_train[["clean_tweet", "label"]]

,clean_tweet,label
0,omg omg omg yay found wonderful price segasatu...,0
1,payintheusa polar bear climb racing angry pola...,0
2,trainhard polar bear climb racing angry polar ...,0
3,turn resignation,1
4,happy bihday hajime hosogai bihday bihday 30,0
...,...,...
22368,fallschurch gorilla simulator need adapt envir...,0
22369,taylor cried cuz found phone emotional times o...,0
22370,factsguide people nowadays motivation,0
22371,mrs reid working p6s today preparation p7s hig...,0


In [6]:
with open('output_train.txt', 'w') as f:
    f.write(df_train['clean_tweet'].str.cat(sep='\n'))

with open('labels_train.txt', 'w') as f:
    f.write(df_train['label'].apply(str).str.cat(sep='\n'))
    
with open('output_test.txt', 'w') as f:
    f.write(df_test['clean_tweet'].str.cat(sep='\n'))
with open('labels_test.txt', 'w') as f:
    f.write(df_test['label'].apply(str).str.cat(sep='\n'))

with open('output_val.txt', 'w') as f:
    f.write(df_val['clean_tweet'].str.cat(sep='\n'))
with open('labels_val.txt', 'w') as f:
    f.write(df_val['label'].apply(str).str.cat(sep='\n'))
    
with open('output_train.txt', 'r') as f:
    tweets_train = f.read()
with open('labels_train.txt', 'r') as f:
    labels_train = f.read()

with open('output_test.txt', 'r') as f:
    tweets_test = f.read()
with open('labels_test.txt', 'r') as f:
    labels_test = f.read()
    
with open('output_val.txt', 'r') as f:
    tweets_val = f.read()
with open('labels_val.txt', 'r') as f:
    labels_val = f.read()

In [7]:
## Split data and put into single words
from collections import Counter

def split_tweet(df):
    tweets_split = df.split('\n')
    all_text2 = ' '.join(tweets_split)
    # create a list of words
    words = all_text2.split()
    # Count all the words using Counter Method
    count_words = Counter(words)

    total_words = len(words)
    sorted_words = count_words.most_common(total_words)
    vocab_to_int = {w:i+1 for i, (w,c) in enumerate(sorted_words)}
    return tweets_split, vocab_to_int

In [8]:
tweets_split_train, _ = split_tweet(tweets_train)
print ('Number of tweets :', len(tweets_split_train))
tweets_split_test, _ = split_tweet(tweets_test) 
tweets_split_val, _ = split_tweet(tweets_val) 

Number of tweets : 22350


In [9]:
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import vocab

tokenizer = get_tokenizer('basic_english')
counter = Counter()
for text in tweets_split_train:
    counter.update(tokenizer(text))
vocab_words = vocab(counter, min_freq=5)

default_index = -1
vocab_words.set_default_index(default_index)

In [10]:
glove_vectors = glove.get_vecs_by_tokens(vocab_words.get_itos())

In [26]:
from torch.nn.utils.rnn import pad_sequence
maxLen = 30
pad_sequence()

In [12]:
import torchtext
MAX_VOCAB_SIZE=10000
BATCH_SIZE =32
BPTT_LENGTH = 32

TEXT = torchtext.legacy.data.Field(lower=True)
PATH = ""
train, val, test = torchtext.legacy.datasets.LanguageModelingDataset.splits(path=PATH, 
    train='output_train.txt', validation='output_val.txt', test='output_test.txt', text_field=TEXT)

TEXT.build_vocab(train, max_size=MAX_VOCAB_SIZE)
VOCAB_SIZE = len(TEXT.vocab)

print(f'Vocabulary size: {VOCAB_SIZE}')

train_iter, val_iter, test_iter = torchtext.legacy.data.BucketIterator.splits(
    (train, val, test), batch_size=32, sort_key=lambda x: len(x.text), repeat=False, shuffle=True)

Vocabulary size: 10002


In [3]:
    # tokenize = lambda x: x.split()
    # TEXT = data.Field(sequential=True, tokenize=tokenize, lower=True, include_lengths=True, batch_first=True, fix_length=200)
    # LABEL = data.LabelField(tensor_type=torch.FloatTensor)
    # train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
    # TEXT.build_vocab(train_data, vectors=GloVe(name='6B', dim=300))
    # LABEL.build_vocab(train_data)

downloading aclImdb_v1.tar.gz


.vector_cache/glove.6B.zip: 862MB [02:40, 5.37MB/s]                           
100%|██████████| 400000/400000 [00:40<00:00, 9900.00it/s] 


In [5]:
# word_embeddings = TEXT.vocab.vectors
# print ("Length of Text Vocabulary: " + str(len(TEXT.vocab)))
# print ("Vector size of Text Vocabulary: ", TEXT.vocab.vectors.size())
# print ("Label Length: " + str(len(LABEL.vocab)))

# train_data, valid_data = train_data.split() # Further splitting of training_data to create new training_data & validation_data
# train_iter, valid_iter, test_iter = data.BucketIterator.splits((train_data, valid_data, test_data), batch_size=32, sort_key=lambda x: len(x.text), repeat=False, shuffle=True)

# vocab_size = len(TEXT.vocab)

Length of Text Vocabulary: 251639
Vector size of Text Vocabulary:  torch.Size([251639, 300])
Label Length: 2


In [ ]:
max_features=100000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(tweets_train))
list_tokenized_train = tokenizer.texts_to_sequences(tweets_train)
list_tokenized_test = tokenizer.texts_to_sequences(tweets_train)

In [ ]:
maxpadlen = 200
X_t=pad_sequences(list_tokenized_train, maxlen=maxpadlen, padding = 'post')
X_te=pad_sequences(list_tokenized_test, maxlen=maxpadlen, padding = 'post')

In [16]:
def get_tweet_words(glove_vector, df):
    train, valid, test = [], [], []
    for i, line in enumerate(df):
        idxs = [glove_vector.stoi[w]        # lookup the index of word
              for w in split_tweet(line)
              if w in glove_vector.stoi] # keep words that has an embedding
        if not idxs: # ignore tweets without any word with an embedding
                continue
        idxs = torch.tensor(idxs) # convert list to pytorch tensor
        label = torch.tensor(int(line[0] == "4")).long()

    return train

train = get_tweet_words(glove, tweets_train)

TypeError: ignored

In [39]:
max_review_length = 30
X_train = sequence.pad_sequences(tweets_split_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(tweets_split_test, maxlen=max_review_length)
X_val = sequence.pad_sequences(tweets_split_val, maxlen=max_review_length)

ValueError: ignored

In [28]:
import torch
import torch.nn as nn


class LSTM(nn.Module):
    """ Container module with an linear encoder/embedding, an RNN module, and a linear decoder.
    """

    def __init__(self, vocab_size, embedding_dim, hidden_dim, n_layers, glove, dropout=0,freeze_glove = False):
        ''' Initialize model parameters corresponding to ---
            - embedding layer
            - linear layer to map from hidden vector to the vocabulary
            - optionally, dropout layers.  Dropout layers can be placed after 
              the embedding layer or/and after the RNN layer. Dropout within
              an RNN is only applied when there are two or more num_layers.
            - optionally, initialize the model parameters.
            
            The arguments are:
            
            vocab_size: size of vocabulary
            embedding_dim: size of an embedding vector
            hidden_dim: size of hidden/state vector in RNN
            num_layers: number of layers in RNN
            dropout: dropout probability.
            
        '''
        super(LSTM, self).__init__()
        self.dropout = nn.Dropout(dropout)
        self.glove = glove_
        self.vocab_size = vocab_size
        self.embed_size = embedding_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.freeze_glove = freeze_glove
        self.word_embeddings = self.embeddings = torch.nn.Embedding.from_pretrained(self.glove, self.freeze_glove)
        self.rnn = nn.LSTM(self.embed_size, self.hidden_dim, self.n_layers, batch_first=True)
        self.hidden2tag = nn.Linear(self.hidden_dim, 1)
        

    def forward(self, input, hidden0):
        ''' 
        Run forward propagation for a given minibatch of inputs using
        hidden0 as the initial hidden state.

        In LSTMs hidden0 = (h_0, c_0). 

        The output of the RNN includes the hidden vector hiddenn = (h_n, c_n).
        Return this as well so that it can be used to initialize the next
        batch.
        
        Unlike previous homework sets do not apply softmax or logsoftmax here, since we'll use
        the more efficient CrossEntropyLoss.  See 
        https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html.
        '''
        batch_size = input.size(0)
        embeds = self.word_embeddings(input)
        print(hidden0[0].shape)
        lstm_out, hidden_new = self.rnn(embeds, hidden0)
#         lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)

        tag_space = self.hidden2tag(lstm_out)
        print(tag_space.shape)
#       sig_out = tag_space.view(batch_size, -1)
        tag_space = tag_space[:, -1]
        print(tag_space.shape)

        return tag_space, hidden_new


    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
#         if (train_on_gpu):
#             hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
#                   weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
#         else:
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden

In [ ]:
# Instantiate the model w/ hyperparams
vocab_size = len(vocab_to_int)+1 # +1 for the 0 padding
output_size = 1
embedding_dim = 400
hidden_dim = 256
n_layers = 2

    
model = LSTM(vocab_size, embedding_dim, hidden_dim, n_layers, glove_=glove_vector, freeze_glove=True)
print(model)

## Try This Model

In [71]:
from torchtext.legacy.data import Field, LabelField, TabularDataset, BucketIterator, Iterator

TEXT = Field(tokenize='spacy',batch_first=True,include_lengths=True)
LABEL = LabelField(dtype = torch.float,batch_first=True)

In [78]:
fields = [('label', LABEL), ('clean_tweet',TEXT)]


In [76]:
training_data=TabularDataset(path = 'hate_train.csv',format = 'csv',fields = fields,skip_header = True)

#print preprocessed text
print(vars(training_data.examples[0]))

{'clean_tweet': ['27857'], 'label': '27857'}


In [23]:
val_data=TabularDataset(path = 'hate_val.csv',format = 'csv',fields = fields,skip_header = True)
test_data=TabularDataset(path = 'hate_test.csv',format = 'csv',fields = fields,skip_header = True)


In [79]:
train, val, test = TabularDataset.splits(path = "/content/", train='train.csv',validation='val.csv',
                                         test='test.csv', format='csv',
                                         fields= fields, skip_header=True)

print(vars(train.examples[0]))

{'label': '0', 'clean_tweet': ['omg', 'omg', 'omg', 'yay', 'found', 'wonderful', 'price', 'segasaturn', 'throwback']}


In [80]:
TEXT.build_vocab(train,min_freq=3,vectors = "glove.twitter.27B.25d")  
LABEL.build_vocab(train)

#No. of unique tokens in text
print("Size of TEXT vocabulary:",len(TEXT.vocab))

#No. of unique tokens in label
print("Size of LABEL vocabulary:",len(LABEL.vocab))

#Commonly used words
print(TEXT.vocab.freqs.most_common(10))  

#Word dictionary
print(TEXT.vocab.stoi)   

Size of TEXT vocabulary: 7559
Size of LABEL vocabulary: 22350
[('love', 1886), ('nt', 1708), ('day', 1617), ('happy', 1163), ('i', 1027), ('m', 839), ('life', 810), ('time', 804), ('like', 737), ('today', 708)]
defaultdict(<bound method Vocab._default_unk_index of <torchtext.legacy.vocab.Vocab object at 0x7f93864468d0>>, {'<unk>': 0, '<pad>': 1, 'love': 2, 'nt': 3, 'day': 4, 'happy': 5, 'i': 6, 'm': 7, 'life': 8, 'time': 9, 'like': 10, 'today': 11, 'new': 12, 'thankful': 13, 'positive': 14, 'get': 15, 'bihday': 16, 'people': 17, 'good': 18, 'ca': 19, 'one': 20, 'see': 21, 'fathers': 22, 'do': 23, 'smile': 24, 's': 25, 'go': 26, 'want': 27, 'take': 28, 'work': 29, 'healthy': 30, 'weekend': 31, 'fun': 32, 'friday': 33, 'got': 34, 'summer': 35, 'family': 36, 'make': 37, 'great': 38, 'best': 39, 'us': 40, 'need': 41, 'way': 42, 'beautiful': 43, 'bull': 44, 'first': 45, 'friends': 46, 'days': 47, 'ur': 48, 'really': 49, 'going': 50, 'wait': 51, 'music': 52, 'morning': 53, 'world': 54, 'you'

In [85]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  

#set batch size
BATCH_SIZE = 64

#Load an iterator
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train, val, test), 
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.clean_tweet),
    sort_within_batch=True,
    device = device)

In [100]:
# for batch in valid_iterator:
#   print(batch)

In [87]:
import torch.nn as nn

class classifier(nn.Module):
    
    #define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout):
        
        #Constructor
        super().__init__()          
        
        #embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        #lstm layer
        self.lstm = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout,
                           batch_first=True)
        
        #dense layer
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        #activation function
        self.act = nn.Sigmoid()
        
    def forward(self, text, text_lengths):
        
        #text = [batch size,sent_length]
        embedded = self.embedding(text)
        #embedded = [batch size, sent_len, emb dim]
      
        #packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths,batch_first=True)
        
        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
        
        #concat the final forward and backward hidden state
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
                
        #hidden = [batch size, hid dim * num directions]
        dense_outputs=self.fc(hidden)

        #Final activation function
        outputs=self.act(dense_outputs)
        
        return outputs

In [88]:
size_of_vocab = len(TEXT.vocab)
embedding_dim = 25
num_hidden_nodes = 2
num_output_nodes = 1
num_layers = 2
bidirection = True
dropout = 0.2

#instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes,num_output_nodes, num_layers, 
                   bidirectional = True, dropout = dropout)

In [89]:
#architecture
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

#Initialize the pretrained embedding
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

print(pretrained_embeddings.shape)

classifier(
  (embedding): Embedding(7559, 25)
  (lstm): LSTM(25, 2, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (fc): Linear(in_features=4, out_features=1, bias=True)
  (act): Sigmoid()
)
The model has 189,572 trainable parameters
torch.Size([7559, 25])


In [90]:
import torch.optim as optim

#define optimizer and loss
optimizer = optim.Adam(model.parameters())
criterion = nn.BCELoss()

#define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    rounded_preds = torch.round(preds)
    
    correct = (rounded_preds == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
#push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

In [96]:
def train(model, iterator, optimizer, criterion):
    
    #initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    #set the model in training phase
    model.train()  
    
    for batch in iterator:
        print(batch)
        #resets the gradients after every batch
        optimizer.zero_grad()   
        
        #retrieve text and no. of words
        text, text_lengths = batch.clean_tweet   
        
        #convert to 1D tensor
        predictions = model(text, text_lengths).squeeze()  
        
        #compute the loss
        loss = criterion(predictions, batch.label)        
        
        #compute the binary accuracy
        acc = binary_accuracy(predictions, batch.label)   
        
        #backpropage the loss and compute the gradients
        loss.backward()       
        
        #update the weights
        optimizer.step()      
        
        #loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [94]:
def evaluate(model, iterator, criterion):
    
    #initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    #deactivating dropout layers
    model.eval()
    
    #deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            #retrieve text and no. of words
            text, text_lengths = batch.clean_tweet
            
            #convert to 1d tensor
            predictions = model(text, text_lengths).squeeze()
            
            #compute loss and accuracy
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            
            #keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [97]:
N_EPOCHS = 5
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    #train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    #evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')


[torchtext.legacy.data.batch.Batch of size 64]
	[.label]:[torch.FloatTensor of size 64]
	[.clean_tweet]:('[torch.LongTensor of size 64x3]', '[torch.LongTensor of size 64]')

[torchtext.legacy.data.batch.Batch of size 64]
	[.label]:[torch.FloatTensor of size 64]
	[.clean_tweet]:('[torch.LongTensor of size 64x5]', '[torch.LongTensor of size 64]')

[torchtext.legacy.data.batch.Batch of size 64]
	[.label]:[torch.FloatTensor of size 64]
	[.clean_tweet]:('[torch.LongTensor of size 64x21]', '[torch.LongTensor of size 64]')

[torchtext.legacy.data.batch.Batch of size 64]
	[.label]:[torch.FloatTensor of size 64]
	[.clean_tweet]:('[torch.LongTensor of size 64x9]', '[torch.LongTensor of size 64]')

[torchtext.legacy.data.batch.Batch of size 64]
	[.label]:[torch.FloatTensor of size 64]
	[.clean_tweet]:('[torch.LongTensor of size 64x13]', '[torch.LongTensor of size 64]')

[torchtext.legacy.data.batch.Batch of size 64]
	[.label]:[torch.FloatTensor of size 64]
	[.clean_tweet]:('[torch.LongTensor of 

KeyError: ignored